In [2]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import time

In [3]:
url= 'https://nigeriapropertycentre.com/for-rent/flats-apartments/lagos/showtype'
headers= {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}

In [4]:
r = requests.get(url, headers=headers)
r.status_code

200

In [5]:
rent = BeautifulSoup(r.text, 'html.parser')

In [6]:
len(rent.contents)
rent.contents[2].name

'html'

In [7]:
titles= rent.select('.content-title')
len(titles)


21

In [8]:
content_title=[]
for title in titles:
    content_title.append(title.get_text(strip=True))
content_title[0]

'Self contained (single rooms) for rent'

In [9]:
content_titles= pd.Series(content_title)

In [10]:
addresses= rent.select('.voffset-bottom-10 > strong ')
len(addresses)

21

In [11]:
content_address=[]
for address in addresses:
    content_address.append(address.get_text(strip=True))
content_address[0]

'Badore, Ajah, Lagos'

In [12]:
content_addresses= pd.Series(content_address)

In [13]:
adds= rent.select('.description.hidden-xs > span ')
len(adds)

20

In [14]:
content_added=[]
for add in adds:
    content_added.append(add.get_text(strip=True))
content_added[0]

'Added Today'

In [15]:
content_addeds = pd.Series(content_added)

In [80]:
prices= rent.select('.price')
len(prices)

42

In [107]:
content_price=[]
for i, price in enumerate(prices):
    if i % 2 !=0:
        content_price.append(price.get('content'))
content_price[0]


'300000.00'

In [18]:
content_prices= pd.Series(content_price)

In [81]:
rent_bedrooms= rent.find_all('i', class_='fa-bed')
bedrooms= []

for rent_bedroom in rent_bedrooms:
    bedrooms.append(rent_bedroom.find_next_sibling('span'))

len(bedrooms)

15

In [82]:
content_bedroom=[]
for bedroom in bedrooms:
    content_bedroom.append(bedroom.get_text(strip=True))
content_bedroom[0]

'3'

In [90]:
rent_bathrooms= rent.find_all('i', class_='fa-bath')
bathrooms= []

for rent_bathroom in rent_bathrooms:
    bathrooms.append(rent_bathroom.find_next_sibling('span'))

len(bathrooms)

21

In [95]:
content_bathroom=[]
for bathroom in bathrooms:
    content_bathroom.append(bathroom.get_text(strip=True))
content_bathroom[2]

'3'

In [99]:
rent_toilets= rent.find_all('i', class_='fa-toilet')
toilets= []

for rent_toilet in rent_toilets:
    toilets.append(rent_toilet.find_next_sibling('span'))

len(toilets)

21

In [102]:
content_toilet=[]
for toilet in toilets:
    content_toilet.append(toilet.get_text(strip=True))
content_toilet[0]

'1'

In [103]:
rent_parking_spaces= rent.find_all('i', class_='fa-car')
parking_spaces= []

for rent_parking_space in rent_parking_spaces:
    parking_spaces.append(rent_parking_space.find_next_sibling('span'))

len(parking_spaces)

16

In [105]:
content_parking=[]
for parking_space in parking_spaces:
    content_parking.append(parking_space.get_text(strip=True))
content_parking[0]

'1'

In [267]:
apartments = pd.DataFrame({'title': content_titles,
                            'address': content_addresses,
                            'date added': content_addeds.reindex(content_titles.index),
                            'price': content_prices})

In [268]:
apartments.to_csv('apartment.csv')

In [108]:
def scrape_page(url):
    """
        Argument: url -> of the website to scrape
        Output: (content_titles, content_addresses, content_addeds, content_prices)

    """
    r = requests.get(url, headers=headers)
    rent = BeautifulSoup(r.text, 'html.parser')
    
    titles= rent.select('.content-title')
    content_title=[]
    for title in titles:
        content_title.append(title.get_text(strip=True))

    
    addresses= rent.select('.voffset-bottom-10 > strong ')
    content_address=[]
    for address in addresses:
        content_address.append(address.get_text(strip=True))
    
    
    adds= rent.select('.description.hidden-xs > span ')
    content_added=[]
    for add in adds:
        content_added.append(add.get_text(strip=True))
    

    rent_bedrooms= rent.find_all('i', class_='fa-bed')
    bedrooms= []
    for rent_bedroom in rent_bedrooms:
        bedrooms.append(rent_bedroom.find_next_sibling('span'))
    content_bedroom=[]
    for bedroom in bedrooms:
        content_bedroom.append(bedroom.get_text(strip=True))
    

    rent_bathrooms= rent.find_all('i', class_='fa-bath')
    bathrooms= []
    for rent_bathroom in rent_bathrooms:
        bathrooms.append(rent_bathroom.find_next_sibling('span'))
    content_bathroom=[]
    for bathroom in bathrooms:
        content_bathroom.append(bathroom.get_text(strip=True))
    

    rent_toilets= rent.find_all('i', class_='fa-toilet')
    toilets= []
    for rent_toilet in rent_toilets:
        toilets.append(rent_toilet.find_next_sibling('span'))
    content_toilet=[]
    for toilet in toilets:
        content_toilet.append(toilet.get_text(strip=True))

    
    rent_parking_spaces= rent.find_all('i', class_='fa-car')
    parking_spaces= []
    for rent_parking_space in rent_parking_spaces:
        parking_spaces.append(rent_parking_space.find_next_sibling('span'))
    content_parking=[]
    for parking_space in parking_spaces:
        content_parking.append(parking_space.get_text(strip=True))


    prices= rent.select('.price')
    content_price=[]
    for i, price in enumerate(prices):
        if i % 2 !=0:
            content_price.append(price.get('content'))
    
    
    return content_title, content_address, content_added, content_bedroom, content_bathroom, content_toilet, content_parking, content_price

In [109]:
#urls =['https://nigeriapropertycentre.com/for-rent/flats-apartments/lagos/showtype', 'https://nigeriapropertycentre.com/for-rent/flats-apartments/lagos/showtype?page=2', 'https://nigeriapropertycentre.com/for-rent/flats-apartments/lagos/showtype?page=3', ]

base_url = 'https://nigeriapropertycentre.com/for-rent/flats-apartments/lagos/showtype'
urls = [base_url]
for i in range(2,55):
    url = base_url + '?page=' + str(i)
    urls.append(url)

In [110]:
master_titles, master_addresses, master_date_added, master_bedrooms, master_bathrooms, master_toilets, master_parking_spaces, master_prices = [], [], [], [], [], [], [], []
for url in urls:
    content_title, content_address, content_added, content_bedroom, content_bathroom, content_toilet, content_parking, content_price = scrape_page(url)
    master_titles += content_title
    master_addresses += content_address
    master_date_added += content_added
    master_bedrooms += content_bedroom
    master_bathrooms += content_bathroom
    master_toilets += content_toilet
    master_parking_spaces += content_parking
    master_prices += content_price

In [111]:
len(master_bedrooms)

951

In [121]:
master_title= pd.Series(master_titles, name= 'title')
master_address= pd.Series(master_addresses, name= 'address')
master_date = pd.Series(master_date_added, name= 'date_added')
master_bedroom = pd.Series(master_bedrooms, name= 'bedroom')
master_bathroom = pd.Series(master_bathrooms, name= 'bathroom')
master_toilet = pd.Series(master_toilets, name= 'toilet')
master_parking_space = pd.Series(master_parking_spaces, name='parking_space')
master_price = pd.Series(master_prices, name= 'price')

In [122]:
lagos_apartments = pd.DataFrame({'title': master_title,
                            'address': master_address,
                            'date_added': master_date,
                            'bedroom': master_bedroom,
                            'bathroom': master_bathroom,
                            'toilet': master_toilet,
                            'parking_space': master_parking_space,
                            'price': master_price})

In [123]:
lagos_apartments.to_csv('lagos_apartments_for_rent.csv')